# 📘 Demo: Using `vdwdata` with `param_sweep_measure`

This notebook shows how to:

- Set up a measurement sweep using `param_sweep_measure`
- Collect and store data using the `vdwdata` package
- Explore the resulting dataset
- Visualize it using raw plots (no interpolation)

⚠️ Assumes you have `vdwdata`, `param_sweep_measure`, `xarray`, `hvplot`, and `panel` installed and importable.


In [1]:
# Imports
import numpy as np
from pathlib import Path

from pyPulses.utils import SweepMeasureProductConfig, sweepMeasureProduct
from ez_data import MeasurementDataset, LiveDashboard, plot_snapped_quadmesh

If you wish to use GPIB instruments, make sure NI488.2 is installed.


## Step 1: Setup Dataset and Sweep Configuration

In [2]:
# Setup dataset path
dataset_path = Path("data/demo_sweep.zarr")
ds = MeasurementDataset(dataset_path)

# Define parameter names
swept_names = ("Vtg", "Vbg")
measured_names = ("resistance",)

# Dummy instrument control
def dummy_set(value): pass
def dummy_get(): return np.random.normal(10_000, 500)

# Callback to collect data
def post_callback(index, coords, data):
    coord_dict = dict(zip(swept_names, coords))
    data_dict = dict(zip(measured_names, data))
    ds.add_point(coord_dict, data_dict)

# Define axes to sweep
axes = (
    np.linspace(-1, 1, 11),  # Vtg
    np.linspace(-0.5, 0.5, 11)  # Vbg
)

# Define config
config = SweepMeasureProductConfig(
    setters=(dummy_set, dummy_set),
    getters=(dummy_get,),
    swept_name=swept_names,
    measured_name=measured_names,
    time_per_point=0.05,
    post_callback=post_callback,
    axes=axes
)

## Step 2: Run the Sweep

In [4]:
sweepMeasureProduct(config)
ds.flush_to_zarr()

c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\zarr\api\asynchronous.py:213: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\zarr\api\asynchronous.py:213: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\zarr\api\asynchronous.py:213: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


## Step 3: Load Data and Plot

## Step 4: Optional - Interactive Dashboard

In [ ]:
dash = LiveDashboard(dataset_path)
dash.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'e6281cdc-d354-467c-a7e7-0133685b2ed6': {'version…

UnknownReferenceError: can't resolve reference '76a27531-652a-4fbb-be9b-d15264be4d02'

UnknownReferenceError: can't resolve reference '76a27531-652a-4fbb-be9b-d15264be4d02'

UnknownReferenceError: can't resolve reference '76a27531-652a-4fbb-be9b-d15264be4d02'

UnknownReferenceError: can't resolve reference '76a27531-652a-4fbb-be9b-d15264be4d02'

...

UnknownReferenceError: can't resolve reference '8fe3b045-2e24-4891-ab02-c1e44b0bf611'

UnknownReferenceError: can't resolve reference 'fdf451d4-eba3-4c32-a7b3-9a97b4ab5646'

UnknownReferenceError: can't resolve reference 'fdf451d4-eba3-4c32-a7b3-9a97b4ab5646'

UnknownReferenceError: can't resolve reference '097e81fc-30cc-4cb4-aab4-2eec5f3ae932'

UnknownReferenceError: can't resolve reference '097e81fc-30cc-4cb4-aab4-2eec5f3ae932'

In [9]:
import xarray as xr
import numpy as np
from typing import Union

def snap_and_grid_for_plotting(
    ds: xr.Dataset,
    x: str,
    y: str,
    z: str,
    resolution: Union[float, dict] = 1e-3,
) -> xr.Dataset:
    """
    Snap x and y coordinates to grid based on specified resolutions and bin z into a grid.

    Parameters:
        ds : xarray.Dataset
        x, y : str - coordinate names
        z : str - variable to grid
        resolution : float or dict - resolution per axis or scalar for both

    Returns:
        gridded Dataset with x, y coords and z values.
    """
    # Support scalar or dict resolution
    if isinstance(resolution, dict):
        res_x = resolution.get(x, 1e-3)
        res_y = resolution.get(y, 1e-3)
    else:
        res_x = res_y = resolution

    x_snapped = (ds[x] / res_x).round() * res_x
    y_snapped = (ds[y] / res_y).round() * res_y

    # Identify all unique snapped values
    x_vals = np.unique(x_snapped)
    y_vals = np.unique(y_snapped)

    x_idx = {v: i for i, v in enumerate(x_vals)}
    y_idx = {v: i for i, v in enumerate(y_vals)}

    grid = np.full((len(y_vals), len(x_vals)), np.nan)

    for i in range(ds.sizes["point"]):
        x_val = float(x_snapped[i].values)
        y_val = float(y_snapped[i].values)
        z_val = float(ds[z][i].values)

        if x_val in x_idx and y_val in y_idx:
            xi = x_idx[x_val]
            yi = y_idx[y_val]
            grid[yi, xi] = z_val

    return xr.Dataset(
        {z: (["y", "x"], grid)},
        coords={"x": x_vals, "y": y_vals}
    )


import hvplot.xarray

def plot_snapped_image(ds, x, y, z, resolution=1e-3):
    gridded = snap_and_grid_for_plotting(ds, x, y, z, resolution)
    return gridded[z].hvplot.image(x="x", y="y", cmap="viridis", colorbar=True)

import xarray as xr
import numpy as np
import random

# Simulate two partial sweeps
def create_fake_sweep(x_range, y_range, noise=0.0003):
    N = len(x_range) * len(y_range)
    xs, ys, zs = [], [], []
    for x in x_range:
        for y in y_range:
            xs.append(x + random.uniform(-noise, noise))
            ys.append(y + random.uniform(-noise, noise))
            zs.append(np.sin(2 * np.pi * x) * np.cos(2 * np.pi * y))
    return xs, ys, zs

# Sweep 1
x1 = np.linspace(-1.0, 0.0, 11)
y1 = np.linspace(-0.5, 0.5, 11)
xs1, ys1, zs1 = create_fake_sweep(x1, y1)

# Sweep 2
x2 = np.linspace(0.01, 1.01, 11)
y2 = np.linspace(-1.0, 0.0, 11)
xs2, ys2, zs2 = create_fake_sweep(x2, y2)

# Combine and create Dataset
xs = np.array(xs1 + xs2)
ys = np.array(ys1 + ys2)
zs = np.array(zs1 + zs2)

ds = xr.Dataset(
    {"signal": (["point"], zs)},
    coords={
        "point": np.arange(len(zs)),
        "Vtg": ("point", xs),
        "Vbg": ("point", ys)
    }
)

def plot_snapped_quadmesh(ds, x, y, z, resolution=1e-3):
    gridded = snap_and_grid_for_plotting(ds, x, y, z, resolution)
    return gridded[z].hvplot.quadmesh(x="x", y="y", cmap="viridis", colorbar=True)

def plot_snapped_quadmesh(ds, x, y, z, resolution=1e-3):
    """
    Plot snapped grid using quadmesh.

    Parameters:
        resolution : float or dict {axis: resolution}
    """
    gridded = snap_and_grid_for_plotting(ds, x, y, z, resolution)
    return gridded[z].hvplot.quadmesh(x="x", y="y", cmap="viridis", colorbar=True)



# Plot with snapping applied (non-destructive)
# plot_snapped_image(ds, x="Vtg", y="Vbg", z="signal", resolution=0.001)
plot_snapped_quadmesh(ds, x="Vtg", y="Vbg", z="signal", resolution={"Vbg":0.01, "Vtg":0.01})


:QuadMesh   [x,y]   (signal)